In [9]:
import json
import re

# Preparation

In [2]:
path_data_prepaired = '../dataset/dataset.json'

In [3]:
data = None
with open(path_data_prepaired) as file_data:
    data = json.load(file_data)

In [25]:
def clear_sentences(data):
    """
        Cleaning sentences, removing special characters and articles
    """
    sentences = list()
    for record in data:
        sentence = record['reviewText']
        sentence = sentence.lower()
        for char in "?.!/;:,":
            sentence = sentence.replace(char, '')

        sentence = sentence.split(sep=' ')
        sentence = [word for word in sentence if len(word) > 1]
        sentences.append(sentence)
        
    return sentences

In [24]:
%time sentences = clear_sentences(data)

CPU times: user 6.67 s, sys: 2.41 s, total: 9.08 s
Wall time: 18.3 s


# Pipeline

In [29]:
def create_vocabulary(sentences, r=200):
    vocabulary = dict()
    word_count = dict()
    num = 0
    
    for sentence in sentences:
        for word in sentence:
            if word not in word_count:
                word_count[word] = 1
            else:
                word_count[word] += 1
    
    for word, count in word_count.items():
        if word_count[word] >= r:
            vocabulary[word] = num
            num += 1
    
    return vocabulary

In [35]:
def create_corpus_matrix(sentences, vocabulary, L = 2):
    words_counts = Counter()
    for sentence_index, sentence in enumerate(sentences):
        for word_index, word in enumerate(sentence):
            if word in vocabulary:
                around_indexes = [i for i in range(max(word_index - L, 0), 
                                                   min(word_index + L + 1, len(sentence))) 
                                  if i != word_index]
                for occur_word_index in around_indexes:
                        occur_word = sentence[occur_word_index]
                        if occur_word in vocabulary:
                            skipgram = (word, occur_word)
                            if skipgram in words_counts:
                                words_counts[skipgram] += 1
                            else:
                                words_counts[skipgram] = 1
    rows = list()
    cols = list()
    values = list()
    
    
    for (word_1, word_2), sharp in words_counts.items():                                            
        rows.append(vocabulary[word_1])
        cols.append(vocabulary[word_2])
        values.append(sharp)
    
    corpus_matrix = sparse.csr_matrix((values, (rows, cols)))
    
    return corpus_matrix

In [32]:
%time vocabulary = create_vocabulary(sentences, r=200)

CPU times: user 4.08 s, sys: 2.04 ms, total: 4.08 s
Wall time: 4.08 s


In [36]:
# Example of prototype embedding_computer function
def embedding_computer(corpus_matrix, vocabulary, **kwargs):
    pass

In [40]:
def compute_embeddings(corpus_matrix, k, d=200):
    """
        PS2 embedding_computer from HW, one more example
    """
    all_observations = corpus_matrix.sum()

    rows = []
    cols = []
    sppmi_values = []

    sum_over_words = np.array(corpus_matrix.sum(axis=0)).flatten()
    sum_over_contexts = np.array(corpus_matrix.sum(axis=1)).flatten()

    for word_index_1, word_index_2 in zip(corpus_matrix.nonzero()[0], 
                                          corpus_matrix.nonzero()[1]):

        sg_count = corpus_matrix[word_index_1, word_index_2]

        pwc = sg_count
        pw = sum_over_contexts[word_index_1]
        pc = sum_over_words[word_index_2]

        spmi_value = np.log2(pwc * all_observations / (pw * pc * k))
        sppmi_value = max(spmi_value, 0)

        rows.append(word_index_1)
        cols.append(word_index_2)
        sppmi_values.append(sppmi_value)

    sppmi_mat = sparse.csr_matrix((sppmi_values, (rows, cols)))
    U, S, V = linalg.svds(sppmi_mat, 200)
    embedding_matrix = U @ scipy.diag(scipy.sqrt(S))
    
    return embedding_matrix

In [41]:
class Word2VecController:
    
    def compute_embeddings(corpus_matrix=None, vocabulary=None, embedding_computer=None):
        """
            @param corpus_matrix - [w_i, c_i], as it was in second PS, 
                                    generated by create_corpus_matrix (maybe it will be 
                                    generated by another function with similar prototype)
            @param vocabulary - it is obvious, what is it
            @param embedding_computer - function, which return embedding matrix
        """
        
        # preparation and execution embeggin_computer function
        # embedding matrix
        
        # self.embedding = ...
        
        return self
    
    def calc_cosine_similarity(word):
        """
            Cosine similarity calculator (maybe from sklearn)
        """
        
        # calculation similarity
        # similarity = cosine(word, self.embedding)
        
        return similarity